In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# This code is part of Qiskit.
#
# (C) Copyright IBM 2021.
#
# This code is licensed under the Apache License, Version 2.0. You may
# obtain a copy of this license in the LICENSE.txt file in the root directory
# of this source tree or at http://www.apache.org/licenses/LICENSE-2.0.
#
# Any modifications or derivative works of this code must retain this
# copyright notice, and modified files need to carry a notice indicating
# that they have been altered from the originals.

In [3]:
"""Rough frequency calibration tests."""
import sys
sys.path.insert(0, r"/home/rupeshknn/projects/qc/pulse-backend-qamp22/qiskit-experiments/test/")

from base import QiskitExperimentsTestCase

import numpy as np

from qiskit.providers.fake_provider import FakeArmonkV2

from qiskit_experiments.framework import BackendData
from qiskit_experiments.library import RoughFrequencyCal
from qiskit_experiments.calibration_management import Calibrations
from qiskit_experiments.calibration_management.basis_gate_library import FixedFrequencyTransmon
from qiskit_experiments.test.mock_iq_backend import MockIQBackend
from qiskit_experiments.test.mock_iq_helpers import MockIQSpectroscopyHelper as SpectroscopyHelper

In [8]:
import unittest
from qiskit_experiments.test.iq_pulse_backend import SingleTransmonTestBackend
pulse_backend = SingleTransmonTestBackend(noise=False)

In [21]:
class TestRoughFrequency(QiskitExperimentsTestCase):
    """Tests for the rough frequency calibration experiment."""
    
    def setUp(self):
        """Setup the tests."""
        super().setUp()
        self.backend = SingleTransmonTestBackend(noise=False)

    def test_init(self):
        """Test that initialization."""
        qubit = 0
        cals = Calibrations.from_backend(self.backend)
        frequencies = [1000, 2000, 3000]
        auto_update = False
        absolute = False

        freq = RoughFrequencyCal(
            qubit, cals, frequencies, auto_update=auto_update, absolute=absolute
        )

        self.assertEqual(freq.physical_qubits, (qubit,))
        self.assertEqual(freq._frequencies, frequencies)
        self.assertEqual(freq._absolute, False)
        self.assertEqual(freq.auto_update, False)

    def test_update_calibrations(self):
        """Test that we can properly update an instance of Calibrations."""
        
        freq01 = BackendData(self.backend).drive_freqs[0]
        backend_5MHz = SingleTransmonTestBackend(qubit_frequency=freq01 + 5e6, noise=False)

#         backend = MockIQBackend(
#             experiment_helper=SpectroscopyHelper(
#                 freq_offset=5e6,
#                 line_width=2e6,
#                 iq_cluster_centers=[((-1.0, -1.0), (1.0, 1.0))],
#                 iq_cluster_width=[0.2],
#             ),
#         )
#         backend._configuration.basis_gates = ["x"]
#         backend._configuration.timing_constraints = {"granularity": 16}

#         backend.defaults().qubit_freq_est = [freq01, freq01]

        library = FixedFrequencyTransmon()
        cals = Calibrations.from_backend(self.backend, libraries=[library])

        prev_freq = cals.get_parameter_value(cals.__drive_freq_parameter__, (0,))
        self.assertEqual(prev_freq, freq01)

        frequencies = np.linspace(freq01 - 10.0e6, freq01 + 10.0e6, 31)
        
        spec = RoughFrequencyCal(0, cals, frequencies)
        spec.set_experiment_options(amp=0.005)
        expdata = spec.run(backend_5MHz)
        
        self.assertExperimentDone(expdata)

        # Check the updated frequency which should be shifted by 5MHz.
        post_freq = cals.get_parameter_value(cals.__drive_freq_parameter__, (0,))
        self.assertTrue(abs(post_freq - freq01 - 5e6) < 1e6)

    def test_experiment_config(self):
        """Test converting to and from config works"""
        cals = Calibrations.from_backend(FakeArmonkV2())
        frequencies = [1, 2, 3]
        exp = RoughFrequencyCal(0, cals, frequencies)
        loaded_exp = RoughFrequencyCal.from_config(exp.config())
        self.assertNotEqual(exp, loaded_exp)
        self.assertTrue(self.json_equiv(exp, loaded_exp))

In [24]:
# obj = TestRoughFrequency("test_update_calibrations")
# obj.setUp()
# obj.test_update_calibrations()

suite = unittest.makeSuite(TestRoughFrequency, 'test')
TestRoughFrequency_result = unittest.TestResult()
suite.run(TestRoughFrequency_result)

/home/rupeshknn/mambaforge/envs/exp/lib/python3.9/site-packages/numpy/lib/function_base.py:2411: RuntimeWarning: invalid value encountered in <lambda> (vectorized)
  outputs = ufunc(*inputs)


<unittest.result.TestResult run=3 errors=0 failures=0>